# Clusterização

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importações

In [ ]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import plotly.express as px
import matplotlib
import time
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('seaborn-darkgrid')
import statsmodels.api as sm
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import altair as alt

In [ ]:
customer = pd.read_csv("/kaggle/input/mall-customers/Mall_Customers.csv")

In [ ]:
customer.head()

### Renomear as variáveis

In [ ]:
customer.rename(columns={"Annual Income (k$)":"AnnualIncome","Spending Score (1-100)":"SScore"}, inplace=True)

## Olhando as variáveis

In [ ]:
source = customer

alt.Chart(source).mark_bar().encode(
    alt.X("Age:Q", bin=True),
    y='count()',
)

In [ ]:
source = customer

alt.Chart(source).mark_bar().encode(
    alt.X("AnnualIncome:Q", bin=True),
    y='count()',
)

In [ ]:
source = customer

alt.Chart(source).mark_bar().encode(
    alt.X("SScore:Q", bin=True),
    y='count()',
)

In [ ]:
customer.Genre.value_counts()

## Montar o Índice

In [ ]:
model = customer.set_index("CustomerID")

In [ ]:
model.info()

In [ ]:
model

## Clientes com correlação

In [ ]:
sns.pairplot(model, hue = 'Genre', diag_kind = 'kde',
             plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
             size = 4)

### Olhar os aggregados da base

In [ ]:
import plotly.express as px

fig = px.scatter_3d(model, x='AnnualIncome', y='SScore', z='Age',
              color='Genre')
fig.show()

In [ ]:
model.groupby(['Genre']).agg({"AnnualIncome": ['mean','median','std'],
                                    "SScore": ['mean','median','std']}).reset_index()

Podemos notar que não há variações expressivas entre gêneros, vamos considerar apenas as 3 outras features

In [ ]:
model_df = model.drop(columns=['Genre']).copy()

## Olhando a distribuição construida

In [ ]:
fig = px.scatter_3d(model, x='AnnualIncome', y='SScore', z='Age')
fig.show()

## Modeling

Vamos começar os modelos, usando o k-means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def kmeans(numero_de_cluster,generos):
    modelo = KMeans(n_clusters=numero_de_cluster)
    modelo.fit(generos)
    return [numero_de_cluster,modelo.inertia_]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
variaveis_escaladas = scaler.fit_transform(model_df)

In [ ]:
resultado = [kmeans(numero_de_grupos,variaveis_escaladas) for numero_de_grupos in range(1,41)]

In [ ]:
resultado = pd.DataFrame(resultado,columns=['grupos','inertia'])

In [ ]:
fig = px.line(resultado, x="grupos",y='inertia')
fig.show()

Podemos notar que os números de elbow para nossa clusterização seriam 6 e 11, vamos construir a clusterização com ambos

In [ ]:
model_df

In [ ]:
model_escalado = model_df.copy()
model_escalado[['Age','AnnualIncome','SScore']] = scaler.fit_transform(model_escalado[['Age','AnnualIncome','SScore']])

In [ ]:
km = KMeans(n_clusters=11)
y_predicted = km.fit_predict(model_escalado[['Age','AnnualIncome','SScore']])
# clustering
model_df['cluster'] = y_predicted

In [ ]:
fig = px.scatter_3d(model_df, x='AnnualIncome', y='SScore', z='Age',
              color='cluster')
fig.show()

Existe uma boa diferenciação dos clusters, mas ainda sim temos dados que são 'mesclados'. Vamos fazer a clusterização com 6 clusters

In [ ]:
km = KMeans(n_clusters=6)
y_predicted = km.fit_predict(model_escalado[['Age','AnnualIncome','SScore']])
# clustering
model_df['cluster'] = y_predicted

In [ ]:
fig = px.scatter_3d(model_df, x='AnnualIncome', y='SScore', z='Age',
              color='cluster')
fig.show()

Podemos notar que a classificação com 6 clustes foi bem mais descritiva. Visualmente temos os clientes dispersos em clusters homogêneos. Vamos manter essa classificação e analisar os dados de cada uma delas

In [ ]:
model_df.groupby(['cluster']).agg({"Age":["mean","median"],
                                   "AnnualIncome":["mean","median"],
                                   "SScore":["mean","median"]}).reset_index()

Vamos fazer uma análise de distribuição das variáveis por cluster

In [ ]:
def bplot (df,variavel):
    plt.figure(figsize=(10,8))
    sns.boxplot(data=df,x="cluster",y=variavel)

In [ ]:
bplot(model_df,"Age")

In [ ]:
bplot(model_df,"AnnualIncome")

In [ ]:
bplot(model_df,"SScore")

Podemos notar que as variáveis estão bem distribuidas entre os clustes. Com poucos ou nenhum outliers e uma construção bastante satisfatória de divisão de público